In [9]:
with open('Lusiadas/lusiadas.txt', 'r', encoding='ISO-8859-1') as f:
    text = f.read()

In [10]:
print('Length of dataset in char: ', len(text))

Length of dataset in char:  318182


In [11]:
print(text[:1000])

1
As armas e os barões assinalados,
Que da ocidental praia Lusitana,
Por mares nunca de antes navegados,
Passaram ainda além da Taprobana,
Em perigos e guerras esforçados,
Mais do que prometia a força humana,
E entre gente remota edificaram
Novo Reino, que tanto sublimaram;

2
E também as memórias gloriosas
Daqueles Reis, que foram dilatando
A Fé, o Império, e as terras viciosas
De África e de Ásia andaram devastando;
E aqueles, que por obras valerosas
Se vão da lei da morte libertando;
Cantando espalharei por toda parte,
Se a tanto me ajudar o engenho e arte.

3
Cessem do sábio Grego e do Troiano
As navegações grandes que fizeram;
Cale-se de Alexandro e de Trajano
A fama das vitórias que tiveram;
Que eu canto o peito ilustre Lusitano,
A quem Neptuno e Marte obedeceram:
Cesse tudo o que a Musa antígua canta,
Que outro valor mais alto se alevanta.

4
E vós, Tágides minhas, pois criado
Tendes em mim um novo engenho ardente,
Se sempre em verso humilde celebrado
Foi de mim vosso rio alegre

In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"'(),-.0123456789:;?ABCDEFGHIJLMNOPQRSTUVXZ[]abcdefghijlmnopqrstuvxyzÀÁÉÍÓÔÚàáâãçèéêíòóôõúü
94


In [13]:
str_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_str = {i:ch for i,ch in enumerate(chars)}

In [16]:
encode = lambda s: [str_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_str[i] for i in l])

In [17]:
encode('sábio Grego')

[65, 80, 49, 56, 61, 1, 29, 64, 52, 54, 61]

In [18]:
decode([65, 80, 49, 56, 61, 1, 29, 64, 52, 54, 61])

'sábio Grego'

In [20]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [21]:
print(data.shape, data.dtype)

torch.Size([318182]) torch.int64


In [22]:
print(data[:1000])

tensor([11,  0, 23, 65,  1, 48, 64, 59, 48, 65,  1, 52,  1, 61, 65,  1, 49, 48,
        64, 91, 52, 65,  1, 48, 65, 65, 56, 60, 48, 58, 48, 51, 61, 65,  7,  0,
        38, 67, 52,  1, 51, 48,  1, 61, 50, 56, 51, 52, 60, 66, 48, 58,  1, 62,
        64, 48, 56, 48,  1, 33, 67, 65, 56, 66, 48, 60, 48,  7,  0, 37, 61, 64,
         1, 59, 48, 64, 52, 65,  1, 60, 67, 60, 50, 48,  1, 51, 52,  1, 48, 60,
        66, 52, 65,  1, 60, 48, 68, 52, 54, 48, 51, 61, 65,  7,  0, 37, 48, 65,
        65, 48, 64, 48, 59,  1, 48, 56, 60, 51, 48,  1, 48, 58, 85, 59,  1, 51,
        48,  1, 41, 48, 62, 64, 61, 49, 48, 60, 48,  7,  0, 27, 59,  1, 62, 52,
        64, 56, 54, 61, 65,  1, 52,  1, 54, 67, 52, 64, 64, 48, 65,  1, 52, 65,
        53, 61, 64, 83, 48, 51, 61, 65,  7,  0, 34, 48, 56, 65,  1, 51, 61,  1,
        63, 67, 52,  1, 62, 64, 61, 59, 52, 66, 56, 48,  1, 48,  1, 53, 61, 64,
        83, 48,  1, 55, 67, 59, 48, 60, 48,  7,  0, 27,  1, 52, 60, 66, 64, 52,
         1, 54, 52, 60, 66, 52,  1, 64, 

In [23]:
split_index = int(0.9*len(data))
train_data = data[:split_index]
val_data = data[split_index:]

In [24]:
context_size = 8
train_data[:context_size+1]

tensor([11,  0, 23, 65,  1, 48, 64, 59, 48])

In [25]:
x = train_data[:context_size]
y = train_data[1:context_size+1]

for t in range(context_size):
    context = x[:t+1]
    target = y[t]
    print(f'context: {context}\ntarget: {target}')

context: tensor([11])
target: 0
context: tensor([11,  0])
target: 23
context: tensor([11,  0, 23])
target: 65
context: tensor([11,  0, 23, 65])
target: 1
context: tensor([11,  0, 23, 65,  1])
target: 48
context: tensor([11,  0, 23, 65,  1, 48])
target: 64
context: tensor([11,  0, 23, 65,  1, 48, 64])
target: 59
context: tensor([11,  0, 23, 65,  1, 48, 64, 59])
target: 48


In [55]:
torch.manual_seed(42)
batch_size = 4
context_size = 8

def get_batch(split):
    data = train_data if split=='train' else val_data
    index = torch.randint(len(data)- context_size, (batch_size,))
    x = torch.stack([data[i:i+context_size] for i in index])
    y = torch.stack([data[i+1:i+context_size+1] for i in index])
    return x,y
    
xb, yb = get_batch('train')
print(f'inputs: {xb.shape}\n{xb}')
print(f'targets: {yb.shape}\n{yb}')

inputs: torch.Size([4, 8])
tensor([[49, 64, 48, 60, 50, 48, 65, 21],
        [48,  1, 54, 52, 60, 66, 52,  0],
        [66, 52, 60, 51, 52, 64, 48, 59],
        [51, 61,  7,  0, 27,  1, 51, 52]])
targets: torch.Size([4, 8])
tensor([[64, 48, 60, 50, 48, 65, 21,  0],
        [ 1, 54, 52, 60, 66, 52,  0, 30],
        [52, 60, 51, 52, 64, 48, 59, 21],
        [61,  7,  0, 27,  1, 51, 52, 62]])


In [56]:
for b in range(batch_size):
    for t in range(context_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'input: {context}\ntarget:{target}')

input: tensor([49])
target:64
input: tensor([49, 64])
target:48
input: tensor([49, 64, 48])
target:60
input: tensor([49, 64, 48, 60])
target:50
input: tensor([49, 64, 48, 60, 50])
target:48
input: tensor([49, 64, 48, 60, 50, 48])
target:65
input: tensor([49, 64, 48, 60, 50, 48, 65])
target:21
input: tensor([49, 64, 48, 60, 50, 48, 65, 21])
target:0
input: tensor([48])
target:1
input: tensor([48,  1])
target:54
input: tensor([48,  1, 54])
target:52
input: tensor([48,  1, 54, 52])
target:60
input: tensor([48,  1, 54, 52, 60])
target:66
input: tensor([48,  1, 54, 52, 60, 66])
target:52
input: tensor([48,  1, 54, 52, 60, 66, 52])
target:0
input: tensor([48,  1, 54, 52, 60, 66, 52,  0])
target:30
input: tensor([66])
target:52
input: tensor([66, 52])
target:60
input: tensor([66, 52, 60])
target:51
input: tensor([66, 52, 60, 51])
target:52
input: tensor([66, 52, 60, 51, 52])
target:64
input: tensor([66, 52, 60, 51, 52, 64])
target:48
input: tensor([66, 52, 60, 51, 52, 64, 48])
target:59
input

In [64]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets= targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits,dim=-1)
            idx_next = torch.multinomial(probs,num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)

torch.Size([32, 94])
tensor(4.8162, grad_fn=<NllLossBackward0>)


torch.Size([32, 94])
tensor(4.8162, grad_fn=<NllLossBackward0>)
